In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import nltk
import numpy as np
import json
from sklearn.model_selection import GridSearchCV
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alici\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def remove_stopwords(text):
    all_words = text.split(" ")
    clean_text = [i for i in all_words if i not in stopwords and i!=""]
    return " ".join(clean_text)

In [4]:
def remove_html_tags(text):
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [5]:
def lower_texts(text):
    return text.lower()

In [6]:
def clean_text(text):
    clean_text = remove_stopwords(text)
    clean_text = remove_html_tags(clean_text)
    clean_text = lower_texts(clean_text)
    return clean_text

In [7]:
def formatTime(seg):
    min = seg//60
    seg = seg % 60
    return str(min)+"min"+str(round(seg, 2))+"s"

In [8]:
with open(r'C:\Users\alici\Documents\tcc\v3\melhores_param\brmoral\gender.json', encoding='utf-8') as f:
    gender_param = json.load(f)
with open(r'C:\Users\alici\Documents\tcc\v3\melhores_param\brmoral\age.json', encoding='utf-8') as f:
    age_param = json.load(f)
with open(r'C:\Users\alici\Documents\tcc\v3\melhores_param\brmoral\age_fem.json', encoding='utf-8') as f:
    age_fem_param = json.load(f)
with open(r'C:\Users\alici\Documents\tcc\v3\melhores_param\brmoral\age_masc.json', encoding='utf-8') as f:
    age_masc_param = json.load(f)

In [9]:
age = []
age_fem = []
age_masc = []
age_sep = []
gender = []

for i in range(10):
    df_train = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\train.csv')
    df_test = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\test.csv')
    
    X_train = df_train['concat'].apply(clean_text).to_numpy()
    X_test = df_test['concat'].apply(clean_text).to_numpy()
    y_train_gender = df_train['GenderClass']
    y_test_gender = df_test['GenderClass']

    tfidfvec_gender = TfidfVectorizer(max_df=0.8)
    tfidf_train_gender = tfidfvec_gender.fit_transform(X_train)
    tfidf_test_gender = tfidfvec_gender.transform(X_test)

    logisticRegr_gender = LogisticRegression(random_state=11, penalty='l2', C=5714.28, solver='lbfgs', multi_class='multinomial')
    logisticRegr_gender.fit(tfidf_train_gender, y_train_gender)
    pred_gender=logisticRegr_gender.predict(tfidf_test_gender)

    f1 = metrics.f1_score(y_test_gender, pred_gender, average='macro')
    gender.append(f1)
    # print(f1)

    y_train_age = df_train['AgeClass']
    y_test_age = df_test['AgeClass']

    tfidfvec_age = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_age = tfidfvec_age.fit_transform(X_train)
    tfidf_test_age = tfidfvec_age.transform(X_test)

    logisticRegr_age = LogisticRegression(random_state=11, penalty=age_param["k_"+str(i+1)]["params"]["penalty"], C=age_param["k_"+str(i+1)]["params"]["C"], solver='liblinear')
    logisticRegr_age.fit(tfidf_train_age, y_train_age)
    pred_age=logisticRegr_age.predict(tfidf_test_age)

    f1 = metrics.f1_score(y_test_age, pred_age, average='macro')
    age.append(f1)

    df_test["PredictGender"] = pred_gender

    df_train_fem = df_train[df_train["GenderClass"]==1]
    df_train_masc = df_train[df_train["GenderClass"]==0]

    df_test_fem = df_test[df_test["PredictGender"]==1]
    df_test_masc = df_test[df_test["PredictGender"]==0]

    X_train_fem = df_train_fem['concat'].apply(clean_text).to_numpy()
    X_test_fem = df_test_fem['concat'].apply(clean_text).to_numpy()
    y_train_fem = df_train_fem['AgeClass']
    y_test_fem = df_test_fem['AgeClass']

    tfidfvec_fem = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_fem = tfidfvec_fem.fit_transform(X_train_fem)
    tfidf_test_fem = tfidfvec_fem.transform(X_test_fem)

    logisticRegr_fem = LogisticRegression(random_state=11, penalty=age_fem_param["k_"+str(i+1)]["params"]["penalty"], C=age_fem_param["k_"+str(i+1)]["params"]["C"], solver='liblinear')
    logisticRegr_fem.fit(tfidf_train_fem, y_train_fem)
    pred_fem=logisticRegr_fem.predict(tfidf_test_fem)

    f1 = metrics.f1_score(y_test_fem, pred_fem, average='macro')
    age_fem.append(f1)

    X_train_masc = df_train_masc['concat'].apply(clean_text).to_numpy()
    X_test_masc = df_test_masc['concat'].apply(clean_text).to_numpy()
    y_train_masc = df_train_masc['AgeClass']
    y_test_masc = df_test_masc['AgeClass']

    tfidfvec_masc = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_masc = tfidfvec_masc.fit_transform(X_train_masc)
    tfidf_test_masc = tfidfvec_masc.transform(X_test_masc)

    logisticRegr_masc = LogisticRegression(random_state=11, penalty=age_masc_param["k_"+str(i+1)]["params"]["penalty"], C=age_masc_param["k_"+str(i+1)]["params"]["C"], solver='liblinear')
    logisticRegr_masc.fit(tfidf_train_masc, y_train_masc)
    pred_masc=logisticRegr_masc.predict(tfidf_test_masc)

    f1 = metrics.f1_score(y_test_masc, pred_masc, average='macro')
    age_masc.append(f1)

    pred_ages_sep = pred_fem.tolist() + pred_masc.tolist()
    y_test = y_test_fem.to_list() +  y_test_masc.to_list()
    age_sep.append(metrics.f1_score(y_test, pred_ages_sep, average='macro'))

print()
print(sum(gender)/len(gender))
print(sum(age)/len(age))
print(sum(age_fem)/len(age_fem))
print(sum(age_masc)/len(age_masc))
print(sum(age_sep)/len(age_sep))

print("\n\n\nvetores")
print(age)
print(age_sep)


0.6211163017213379
0.37952834586428236
0.4285291070585188
0.4525282730536186
0.46874563466362734



vetores
[0.45391691903319814, 0.4368647824025398, 0.4223737638670319, 0.4047877145438121, 0.3671419929133177, 0.18095238095238095, 0.4351921470342523, 0.18095238095238095, 0.4644820258855346, 0.4486193510583754]
[0.4910554561717353, 0.4555667902482867, 0.41111111111111104, 0.37423510466988724, 0.4989035087719298, 0.48657407407407405, 0.4457121825542878, 0.5423280423280423, 0.5936507936507937, 0.3883192830561251]


In [10]:
np.std(age), np.std(age_sep)

(0.10266561997685311, 0.06493638940646689)

In [11]:
from scipy.stats import wilcoxon
wilcoxon(age, age_sep, alternative='greater')

WilcoxonResult(statistic=12.0, pvalue=0.947265625)

In [12]:
# literatura
gender = []
age = []
age_fem = []
age_masc = []
age_sep = []

for i in range(10):
    df_train = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\train.csv')
    df_test = pd.read_csv(r'C:\Users\alici\Documents\tcc\v3\particoes\brmoral\k_'+str(i+1)+r'\test.csv')
      
    X_train = df_train['concat'].apply(clean_text).to_numpy()
    X_test = df_test['concat'].apply(clean_text).to_numpy()

    tfidfvec_gender = TfidfVectorizer(max_df=0.8)
    tfidf_train_gender = tfidfvec_gender.fit_transform(X_train)
    tfidf_test_gender = tfidfvec_gender.transform(X_test)

    f1 = metrics.f1_score(y_test_gender, pred_gender, average='macro')
    gender.append(f1)

    y_train_age = df_train['AgeClass']
    y_test_age = df_test['AgeClass']

    tfidfvec_age = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_age = tfidfvec_age.fit_transform(X_train)
    tfidf_test_age = tfidfvec_age.transform(X_test)

    logisticRegr_age = LogisticRegression(penalty="l2", C=2857.14, solver='liblinear')
    logisticRegr_age.fit(tfidf_train_age, y_train_age)
    pred_age=logisticRegr_age.predict(tfidf_test_age)

    f1 = metrics.f1_score(y_test_age, pred_age, average='macro')
    age.append(f1)

    df_train_fem = df_train[df_train["GenderClass"]==1]
    df_train_masc = df_train[df_train["GenderClass"]==0]

    df_test_fem = df_test[df_test["GenderClass"]==1]
    df_test_masc = df_test[df_test["GenderClass"]==0]

    X_train_fem = df_train_fem['concat'].apply(clean_text).to_numpy()
    X_test_fem = df_test_fem['concat'].apply(clean_text).to_numpy()
    y_train_fem = df_train_fem['AgeClass']
    y_test_fem = df_test_fem['AgeClass']

    tfidfvec_fem = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_fem = tfidfvec_fem.fit_transform(X_train_fem)
    tfidf_test_fem = tfidfvec_fem.transform(X_test_fem)

    logisticRegr_fem = LogisticRegression(penalty="l2", C=2857.14, solver='liblinear')
    logisticRegr_fem.fit(tfidf_train_fem, y_train_fem)
    pred_fem=logisticRegr_fem.predict(tfidf_test_fem)

    f1 = metrics.f1_score(y_test_fem, pred_fem, average='macro')
    age_fem.append(f1)

    X_train_masc = df_train_masc['concat'].apply(clean_text).to_numpy()
    X_test_masc = df_test_masc['concat'].apply(clean_text).to_numpy()
    y_train_masc = df_train_masc['AgeClass']
    y_test_masc = df_test_masc['AgeClass']

    tfidfvec_masc = TfidfVectorizer(max_features = 1000, max_df=0.8)
    tfidf_train_masc = tfidfvec_masc.fit_transform(X_train_masc)
    tfidf_test_masc = tfidfvec_masc.transform(X_test_masc)

    logisticRegr_masc = LogisticRegression(penalty="l2", C=2857.14, solver='liblinear')
    logisticRegr_masc.fit(tfidf_train_masc, y_train_masc)
    pred_masc=logisticRegr_masc.predict(tfidf_test_masc)

    f1 = metrics.f1_score(y_test_masc, pred_masc, average='macro')
    age_masc.append(f1)

    pred_ages_sep = pred_fem.tolist() + pred_masc.tolist()
    y_test = y_test_fem.to_list() +  y_test_masc.to_list()
    age_sep.append(metrics.f1_score(y_test, pred_ages_sep, average='macro'))

print()
print(sum(gender)/len(gender))
print(sum(age)/len(age))
print(sum(age_fem)/len(age_fem))
print(sum(age_masc)/len(age_masc))
print(sum(age_sep)/len(age_sep))


0.6386639676113361
0.4490891653331991
0.4202769452769453
0.48188831485208306
0.47026960833865167


In [13]:
np.std(age), np.std(age_sep)

(0.03553859749632961, 0.06220541154228602)

In [14]:
from scipy.stats import wilcoxon
wilcoxon(age, age_sep, alternative='greater')

WilcoxonResult(statistic=18.0, pvalue=0.8388671875)